In [24]:
from bertopic import BERTopic
import pandas as pd
import pickle
from sklearn.datasets import fetch_20newsgroups#
import datetime as dt
import numpy as np

In [16]:
submissions1 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions1.json")
submissions2 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions2.json")
submissions3 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions3.json")
submissions4 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions4.json")
submissions5 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions5.json")
submissions6 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions6.json")
submissions7 = pd.read_json("/Users/ritikabrahmadesam/Documents/College/RCOS/OPTUS-AI_RAWDATA/opiates_submissions7.json")

In [17]:
submissions = pd.concat([submissions1, submissions2, submissions3, submissions4, submissions5, submissions6, submissions7])

In [18]:
#create a column with real datetimes
submissions['created_utc'] = submissions['created_utc'].apply(lambda date: dt.datetime.fromtimestamp(date))
submissions = submissions.sort_values(by='created_utc', ascending=True)

In [19]:
posts = submissions[submissions['created_utc'].dt.year==2011]
# curr_df = {"january": posts[posts['created_utc'].dt.month==1],
#            "febuary": posts[posts['created_utc'].dt.month==2],
#            "march": posts[posts['created_utc'].dt.month==3],
#             "april": posts[posts['created_utc'].dt.month==4],
#             "may": posts[posts['created_utc'].dt.month==5],
#             "june": posts[posts['created_utc'].dt.month==6],
#             "august": posts[posts['created_utc'].dt.month==8],
#             "september": posts[posts['created_utc'].dt.month==9],
#             "october": posts[posts['created_utc'].dt.month==10],
#             "november": posts[posts['created_utc'].dt.month==11],
#             "december": posts[posts['created_utc'].dt.month==12]}


In [20]:
#docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.Series(docs['data']).sample(5000).to_list() # huely cut down the data because it takes forever otherwise. This is slow but w/e
data[:5]

["\nThat is absolutely correct.  They have a streak of meanness that runs much\ndeeper and stronger than anything I ever experienced even during the height\nof the 60's racial struggles.  I am absolutely convinced that there are\nkids out there today that have no concept of right and wrong, but have the\nhuman intelligence which is turning them into the greatest predators that\nhave ever walked the earth.  They will prevail unless the rest of humanity\ndecides that it is in their best interest to stand up against these feral\nhumans, and for individuals to start taking some responsibility for their\nown protection.  In a state in which the individuals turn to the authorities\nand police for protection, the police and government is soon composed of\nthe very feral humans that they originally sought protection from.\n\nGee, I guess you touched my hot button.  I'd better go cool off somewhere.\n\n",
 "Reply address: mark.prado@permanet.org\n\n > From: higgins@fnalf.fnal.gov (Bill Higgins-

In [21]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
if 0:
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
    embeddings = embedding_model.encode(data, show_progress_bar=True)
    
    with open("embeddings.pkl", "wb") as fOut:
        pickle.dump({'embeddings': embeddings},fOut)

In [22]:
# Load embeddings from file
embeddings = None
if 1:
    with open("embeddings.pkl", "rb") as fIn:
        cache_data = pickle.load(fIn)
        embeddings = cache_data['embeddings']

In [23]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-11-19 13:16:44,777 - BERTopic - Reduced dimensionality
2023-11-19 13:16:44,977 - BERTopic - Clustered reduced embeddings


In [10]:
# A maybe better representation?
from bertopic.representation import KeyBERTInspired
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired()) 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-11-19 13:10:05,717 - BERTopic - Reduced dimensionality
2023-11-19 13:10:05,934 - BERTopic - Clustered reduced embeddings


In [11]:
topicModel.save("keyBertModel", serialization='safetensors')

In [12]:
topicModel.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1423,-1_think_what_about_much,"[think, what, about, much, just, have, don, no...","[In an earlier article, I explained that what ..."
1,0,462,0_anonymous_posting_what_other,"[anonymous, posting, what, other, re, are, mor...",[[to Benedikt Roseneau ]\n\n#In article <1qv6a...
2,1,414,1_wiring_wire_get_than,"[wiring, wire, get, than, if, should, have, bu...","[\n\nYup, you said it. I admire such honesty...."
3,2,317,2_stephanopoulos_president_what_don,"[stephanopoulos, president, what, don, are, as...","[\n\nSo, like, what do you do during those six..."
4,3,252,3_on_ve_mj_have,"[on, ve, mj, have, just, mz, what, re, no, if]","[#|> #\n#|> #Noting that a particular society,..."
5,4,210,4_enclaves_tartar_tartars_magi,"[enclaves, tartar, tartars, magi, who, labour,...",[From: Center for Policy Research <cpr>\nSubje...
6,5,184,5_stephanopoulos_ra_god_something,"[stephanopoulos, ra, god, something, jesus, wh...",[Rick Anderson replied to my letter with...\n\...
7,6,156,6_information_windows_about_system,"[information, windows, about, system, what, ar...",[\nBut a disk compare utility (old versus new)...
8,7,104,7_gazans_gaza_israeli_israel,"[gazans, gaza, israeli, israel, jews, armenian...","[: Ok, a few days back, the below-included mes..."
9,8,96,8_ripem_greek_authorities_who,"[ripem, greek, authorities, who, what, public,...",[\n\nDr. Goebels thought that a lie repeated e...


In [13]:
topicModel.visualize_topics()